<a href="https://colab.research.google.com/github/seoyen1122/solar_rag/blob/main/mmlu_pro/astute_rag_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

문서 분석 로직 & 외부 문서는 그냥 불러오기


In [ ]:
pip install langchain-upstage langchain-community pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.8/473.8 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: langchain-core


In [ ]:
pip install wikipedia-api

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.8.1-py3-none-any.whl size=15383 sha256=30be5a29aaa43a4f201b5e74eab29551781082e7f5aeb64ddc7653b97b98a1ec
  Stored in directory: /root/.cache/pip/wheels/33/3c/79/b36253689d838af4a0539782853ac3cc38a83a6591ad570dde
Successfully built wikipedia-api


In [ ]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 74.3 MB/s eta 0:00:00


In [ ]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 27.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-upstage 0.7.5 requires tokenizers<0.21.0,>=0.20.0, but you have tokenizers 0.22.1 which is incompatible.


In [ ]:
import os
import json
from typing import List, Tuple

from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate

import wikipediaapi
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/nlp team project

/content/drive/MyDrive/nlp team project


In [ ]:
UPSTAGE_API_KEY = "up_VYzFNHEoEJPfAwYUNp5v9n1CPnMOm"

In [ ]:
df = pd.read_csv("testset.csv")

In [ ]:
llm_classifier = ChatUpstage(
    api_key=UPSTAGE_API_KEY,
    model="solar-pro2",
    temperature=0.1, #거의 항상 같은 결과, 안정적이게
)

In [ ]:
llm_solver = ChatUpstage(
    api_key=UPSTAGE_API_KEY,
    model="solar-pro2",
    temperature=0.1,
)

In [ ]:
llm_solar_pro = ChatUpstage(
    api_key=UPSTAGE_API_KEY,
    model="solar-pro2",
    temperature=0.1, #거의 항상 같은 결과, 안정적이게
)

In [ ]:
category_index = {
    "law":        "/content/drive/MyDrive/nlp team project/code/mmlu_category/law",
    "psychology": "/content/drive/MyDrive/nlp team project/code/mmlu_category/psychology",
    "business":   "/content/drive/MyDrive/nlp team project/code/mmlu_category/business",
    "philosophy": "/content/drive/MyDrive/nlp team project/code/mmlu_category/philosophy",
    "history":    "/content/drive/MyDrive/nlp team project/code/mmlu_category/history",
}

categories = list(category_index.keys())

In [ ]:

emb = UpstageEmbeddings(api_key=UPSTAGE_API_KEY, model="solar-embedding-1-large-passage")

# 카테고리별 faiss 한번에 로드해서 캐시
vectorstore = {}
for cat, path in category_index.items():
    vectorstore[cat] = FAISS.load_local(
        folder_path=path,
        embeddings=emb,
        allow_dangerous_deserialization=True,
    )

wiki = wikipediaapi.Wikipedia(user_agent= "NLP-RAG/1.0", language = 'en')

**1st LLM**:

- return category

- return 3 keywords

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
import json

# 질문 분석을 위한 전용 프롬프트
P_ANALYZE_TMPL = """
You are an expert Question Analyst for MMLU-Pro.
Your task is NOT to answer the question, but to dissect it so that a researcher can find the best evidence.

[Task]
Analyze the following exam question and extracted information:

Question:
{question_text}

Options:
{options_text}

Output a JSON object with the following fields:
1. "core_intent": Summarize what the question is asking in 1 sentence.
2. "constraints": A list of strict conditions that the answer MUST satisfy (e.g., time period, specific person, negation 'NOT', location).
3. "search_queries": A list of 3-5 optimized search queries to put into a vector database or Wikipedia.
   - Break down complex questions into simpler sub-queries.
   - Include specific entities mentioned in the question.

Example Output:
{{
  "core_intent": "Identify the specific treaty that ended the Crimean War.",
  "constraints": ["Must be a treaty", "Related to Crimean War", "Must match the historical timeline (1856)"],
  "search_queries": ["Treaty ending Crimean War", "Paris Treaty 1856", "Crimean War conclusion diplomatic agreement"]
}}

Return ONLY the valid JSON object.
""".strip()

p_analyze_prompt = ChatPromptTemplate.from_template(P_ANALYZE_TMPL)

def run_p_analyze(question_text: str, options: dict, llm=None) -> dict:
    """
    질문을 분석하여 검색 쿼리와 제약 조건을 추출합니다.
    """
    if llm is None:
        llm = llm_solver  # 기존에 정의된 llm 객체 사용

    # 옵션 텍스트화
    options_str = "\n".join([f"({k}) {v}" for k, v in options.items()])

    messages = p_analyze_prompt.format_messages(
        question_text=question_text,
        options_text=options_str
    )

    resp = llm.invoke(messages)
    raw = resp.content.strip()

    # JSON 파싱 (실패 시 기본값 반환 처리 포함)
    try:
        # 마크다운 코드 블록 제거 등 기본적인 정제
        if "```json" in raw:
            raw = raw.split("```json")[1].split("```")[0].strip()
        elif "```" in raw:
            raw = raw.split("```")[0].strip()

        analysis = json.loads(raw)
        return analysis
    except Exception as e:
        print(f"[WARN] p_analyze failed: {e}")
        # 실패 시 기본값: 검색어는 질문 그대로
        return {
            "core_intent": "Unknown",
            "constraints": [],
            "search_queries": [question_text]
        }

In [ ]:
category_prompt_template = """
You are an expert exam classifier for MMLU-Pro.

There are 5 possible categories:
- law
- psychology
- business
- philosophy
- history

[Task]
You will be given ONE multiple-choice exam question (stem + options).

First, think about the question in the following way:

1. Read the entire question and options carefully.
2. Extract 3–6 essential CLUES:
   - short phrases capturing key entities, concepts, time periods, theories, or legal doctrines.
   - Each clue should be specific enough to guide retrieval.

Then, based on these clues:

3. Choose the single best category from the 5 above.
4. Propose 5 EXACT Wikipedia article titles for retrieval.
   - Titles 1–2: Specific entities / proper nouns
     (e.g., "Sherman Antitrust Act", "Jean Piaget").
   - Titles 3–5: Broader parent topics
     (e.g., "Antitrust law", "Developmental psychology").
   - [CRITICAL] Do NOT use descriptive phrases like
     "history of...", "principles of...". Just concise noun titles.

[Output]
Return ONLY one valid JSON object with fields:
- "clues": list of 3–6 short strings
- "category": one of ["law", "psychology", "business", "philosophy", "history"]
- "keywords": list of exactly 5 strings (Wikipedia article titles)

[Example]
Q: "In 1797, John Frere made a discovery..."
Output:
{{
  "clues": [
    "1797 discovery",
    "John Frere",
    "ancient stone tools",
    "Paleolithic archaeology"
  ],
  "category": "history",
  "keywords": [
    "John Frere",
    "Hoxne Hoard",
    "Paleolithic",
    "Stone Age",
    "Archaeology"
  ]
}}

Now classify the following question and return ONLY the JSON object.

[Question]
{question}
""".strip()

# Re-create the prompt object
category_prompt = ChatPromptTemplate.from_template(category_prompt_template)

In [ ]:
def classify_and_extract_keywords(question_text: str):
    # 위에서 정의한 (Few-Shot이 추가된) 프롬프트 템플릿 사용
    messages = category_prompt.format_messages(question=question_text)

    # LLM 호출 (temperature=0.1 정도로 낮게 설정된 llm_classifier 권장)
    resp = llm_classifier.invoke(messages)
    raw = resp.content.strip()

    # 빈 응답 방어 로직
    if not raw:
        print("[WARN] Empty LLM output for category, fallback to 'history'")
        return "history", ["history", "war", "europe"]

    # JSON 파싱 로직
    data = {}
    try:
        # 1차 시도: 전체 파싱
        data = json.loads(raw)
    except json.JSONDecodeError:
        # 2차 시도: 마크다운 코드블록(```json ... ```)이나 잡다한 텍스트 제거 후 파싱
        start = raw.find("{")
        end = raw.rfind("}")
        if start != -1 and end != -1:
            json_str = raw[start:end+1]
            try:
                data = json.loads(json_str)
            except json.JSONDecodeError as e:
                print(f"[WARN] JSON parse failed: {e}")
        else:
            print("[WARN] No JSON braces found.")

    # 파싱 실패 시 기본값 반환
    if not data:
         return "history", ["history", "war", "europe"]

    # 데이터 추출
    category = data.get("category", "history").strip().lower()
    raw_keywords = data.get("keywords", [])

    # [수정 포인트] 문자열 변환 및 공백 제거만 하고, 개수 제한은 [:5] 정도로 넉넉하게
    keywords = [str(k).strip() for k in raw_keywords if str(k).strip()][:5]

    # 키워드가 하나도 없으면 기본값
    if not keywords:
        keywords = ["history", "war", "europe"]

    return category, keywords

In [ ]:
def fetch_wiki_context(keywords, window_size=2000):
    snippets = []
    seen_titles = set()

    for kw in keywords:
        kw = (kw or "").strip()
        if not kw: continue

        candidates = [kw, kw.title()]
        # 괄호 제거 등 추가 처리
        if "(" in kw: candidates.append(kw.split("(")[0].strip())

        found_flag = False
        for cand in candidates:
            if cand in seen_titles: continue

            try:
                page = wiki.page(cand)
                if page.exists():
                    seen_titles.add(cand)
                    full_text = page.text

                    # [개선] 단순히 앞부분만 자르는 게 아니라, 키워드가 등장하는 위치를 찾음
                    # 키워드가 텍스트 내에 있으면 그 주변을 가져옴. 없으면 앞부분 가져옴.
                    lower_text = full_text.lower()
                    lower_kw = kw.lower()
                    start_idx = lower_text.find(lower_kw)

                    if start_idx == -1:
                        # 키워드 못 찾으면 그냥 앞부분
                        display_text = full_text[:window_size]
                    else:
                        # 키워드 주변부 (앞으로 500자, 뒤로 1500자 정도)
                        start_pos = max(0, start_idx - 500)
                        end_pos = min(len(full_text), start_idx + window_size)
                        display_text = full_text[start_pos:end_pos]

                    clean_text = display_text.replace('\n', ' ')
                    snippets.append(f"[Wikipedia: {page.title}]\n...{clean_text}...")
                    found_flag = True
                    break
            except Exception:
                continue

    if not snippets:
        return ""

    return "\n\n".join(snippets)

In [ ]:
import re
import json
from typing import List, Dict, Any

import pandas as pd
from langchain_core.prompts import ChatPromptTemplate

############################################
# 0. 상수 & 기본 설정
############################################

TOP_K_TEXTBOOK = 5
TOP_K_WIKI = 5

In [ ]:
############################################
# 1. 질문 + 옵션(A~I) 파싱
############################################

# 1. 질문 + 옵션(A~J) 파싱
def parse_question_and_options(prompt: str):
    """
    prompt: QUESTION + (A) ~ (J) 옵션이 한 문자열에 들어있는 형태
    return: question_text(str), options(dict: 'A'~'J' -> str)
    """
    pattern = r"\(([A-J])\)\s*"
    matches = list(re.finditer(pattern, prompt))

    if not matches:
        return prompt.strip(), {}

    first = matches[0]
    question_text = prompt[:first.start()].strip()

    options: Dict[str, str] = {}
    for idx, m in enumerate(matches):
        letter = m.group(1)  # 'A'~'J'
        start = m.end()
        end = matches[idx + 1].start() if idx + 1 < len(matches) else len(prompt)
        opt_text = prompt[start:end].strip()
        options[letter] = opt_text

    return question_text, options


In [ ]:
############################################
# 2. "Final Answer: X" 파서
############################################

def extract_final_answer_letter(text: str) -> str:
    """
    p_ans 출력에서 'Final Answer: X' 의 X(A~J)를 뽑기.
    """
    m = re.search(r"Final Answer:\s*([A-J])\s*$", text, flags=re.IGNORECASE | re.MULTILINE)
    if m:
        return m.group(1).upper()
    return ""

In [ ]:

############################################
# 3. Memory p_gen (INTERNAL 문서 생성)
############################################

P_MEM_TMPL = """
You are a knowledgeable expert in {category}.

Task: Using ONLY your own internal knowledge (without seeing any external documents),
generate a short document that provides accurate and relevant information to help
answer the given exam question.

If you truly do not know the answer or lack enough information, explicitly state
"I don't know" rather than hallucinating facts.

Exam Question:
{question_text}

Write ONE coherent document that:
- Focuses only on concepts, definitions, and relations that are relevant to the question.
- Is self-contained and clear enough to help another model answer the question.
- Avoids unnecessary details.

Document:
""".strip()

p_mem_prompt = ChatPromptTemplate.from_template(P_MEM_TMPL)


def run_p_mem(category: str, question_text: str, llm=None) -> str:
    if llm is None:
        llm = llm_solver
    msgs = p_mem_prompt.format_messages(
        category=category,
        question_text=question_text,
    )
    resp = llm.invoke(msgs)
    return resp.content.strip()

In [ ]:
def run_p_gen_for_docs(
    category: str,             # 사용 안 함 (호환성 위해 남겨둠)
    source_type: str,          # "TEXTBOOK" or "WIKIPEDIA"
    question_text: str,        # 사용 안 함 (호환성 위해 남겨둠)
    raw_passages: List[str],   # 검색된 원본 텍스트 리스트
    start_doc_idx: int = 1,
    llm=None,                  # 사용 안 함 (호환성 위해 남겨둠)
) -> Tuple[str, int]:
    """
    [수정됨] LLM을 통한 요약/재작성을 수행하지 않습니다.
    검색된 Raw Passage를 그대로 포맷팅하여 반환합니다.
    이렇게 해야 정보 손실(Information Loss) 없이 p_con이나 p_ans 단계로 넘어갑니다.
    """
    doc_blocks = []
    cur_idx = start_doc_idx

    for k, passage in enumerate(raw_passages, start=1):
        # 빈 텍스트 무시
        if not passage or not passage.strip():
            continue

        # [중요] LLM 호출(invoke) 없이 원본 텍스트를 그대로 넣습니다.
        # 필요하다면 너무 긴 문단만 파이썬 문자열 슬라이싱으로 자르세요 (예: [:2000])
        clean_passage = passage.strip()

        block = (
            f"[Doc {cur_idx} | SOURCE=EXTERNAL({source_type}) | ORIG_ID={source_type}_{k}]\n"
            f"{clean_passage}"
        )
        doc_blocks.append(block)
        cur_idx += 1

    return "\n\n".join(doc_blocks), cur_idx

In [ ]:
!pip install rank_bm25

In [ ]:
import numpy as np
from typing import List
from rank_bm25 import BM25Okapi

############################################
# 5. Retrieval helpers (Hybrid: Vector + BM25)
############################################

# 전역 캐시 (BM25 인덱스를 매번 만들지 않기 위해 저장)
bm25_store = {}   # {category: BM25Okapi_object}
doc_store = {}    # {category: [text_list]}

def init_bm25_for_category(category: str):
    """
    [초기화] 해당 카테고리의 모든 문서를 로드하여 BM25 인덱스를 생성
    """
    if category in bm25_store:
        return

    if category not in vectorstore:
        return

    print(f"[Init] Building BM25 index for: {category}...")
    vs = vectorstore[category]

    try:
        # FAISS 메모리에서 모든 문서 텍스트 추출
        # (주의: FAISS 로드 시 데이터가 메모리에 있어야 함)
        all_docs = list(vs.docstore._dict.values())
        texts = [d.page_content for d in all_docs]

        if not texts:
            print(f"[Warn] No texts found in docstore for {category}.")
            return

        # 토크나이징 (단순 띄어쓰기 기준)
        tokenized_corpus = [doc.lower().split() for doc in texts]

        # 저장
        bm25_store[category] = BM25Okapi(tokenized_corpus)
        doc_store[category] = texts
        print(f"[Init] BM25 ready for {category} ({len(texts)} docs)")

    except Exception as e:
        print(f"[Warn] Failed to init BM25 for {category} (Pure Vector mode will be used): {e}")


def reciprocal_rank_fusion(results_list: List[List[str]], k=60) -> List[str]:
    """
    [RRF 알고리즘] 여러 검색 결과의 순위를 합산하여 재정렬
    """
    fused_scores = {}

    for docs in results_list:
        for rank, doc_text in enumerate(docs):
            if doc_text not in fused_scores:
                fused_scores[doc_text] = 0
            # 순위가 높을수록(rank가 작을수록) 높은 점수 부여
            fused_scores[doc_text] += 1 / (rank + k)

    # 점수 높은 순 정렬
    reranked = sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    return [text for text, score in reranked]


def get_textbook_passages(category: str, query: str) -> List[str]:
    """
    [수정된 함수] Hybrid Search 적용
    1. Vector Search (의미 검색) -> 10개
    2. BM25 Search (키워드 검색) -> 10개
    3. RRF Fusion -> 상위 5개 반환
    """
    if category not in vectorstore:
        return []

    # 0. BM25 준비
    if category not in bm25_store:
        init_bm25_for_category(category)

    # 1. Vector Search (Semantic)
    vs = vectorstore[category]
    # 후보를 10개 정도 가져옴
    vec_candidates = vs.similarity_search(query, k=10)
    vec_results = [d.page_content for d in vec_candidates]

    # 2. BM25 Search (Keyword) - 만약 준비 안됐으면 생략
    bm25_results = []
    if category in bm25_store:
        bm25 = bm25_store[category]
        docs = doc_store[category]

        tokenized_query = query.lower().split()
        # 키워드 매칭 상위 10개
        bm25_results = bm25.get_top_n(tokenized_query, docs, n=10)

    # 3. Fusion (하나만 있으면 그것만 반환)
    if not bm25_results:
        final_passages = vec_results
    else:
        # 두 결과를 섞음
        final_passages = reciprocal_rank_fusion([vec_results, bm25_results], k=60)

    # 최종 TOP_K 개수만큼 자르기
    return final_passages[:TOP_K_TEXTBOOK]


def get_wiki_passages(keywords: List[str]) -> List[str]:
    """
    (기존 유지)
    """
    try:
        raw = fetch_wiki_context(keywords)
    except Exception as e:
        print(f"[WARN] Wikipedia retrieval failed: {e}")
        return []

    raw = (raw or "").strip()
    if not raw:
        return []
    return [raw]

In [ ]:
############################################
# 6. Iterative Knowledge Consolidation p_con (Few-Shot Optimized)
############################################

P_CON_TMPL = """
You are a Knowledge Integrator for MMLU-Pro dataset.

Task: Consolidate information from both your own memorized documents (INTERNAL)
and externally retrieved documents (EXTERNAL) in response to the given exam question.

Guidelines:
* For documents that provide consistent information, cluster them together and
  summarize the key details into a single, concise document.
* For documents with conflicting information, separate them into distinct documents,
  ensuring each document captures the unique perspective or data.
* Exclude any information that is irrelevant to the question.
* For each NEW document you create, clearly indicate:
  - Whether the source was INTERNAL(MEMORY) or EXTERNAL(TEXTBOOK/WIKIPEDIA).
  - The original document numbers that contributed to it (e.g., FROM=1,3,4).

[Examples]

<Example 1: Conflict Resolution>
Initial Context:
[Doc 1 | SOURCE=INTERNAL(MEMORY)]
The Treaty of Versailles was signed in 1918.
[Doc 2 | SOURCE=EXTERNAL(TEXTBOOK)]
The Treaty of Versailles was signed on June 28, 1919, officially ending WWI.

Question: When was the Treaty of Versailles signed?

New Context:
[Doc 1 | SOURCE=EXTERNAL(TEXTBOOK) | FROM=2]
The Treaty of Versailles was signed on June 28, 1919.
(Correction: Internal memory incorrectly stated 1918, which was the armistice year, not the treaty signing.)
</Example 1>

<Example 2: Merging Consistent Info>
Initial Context:
[Doc 1 | SOURCE=EXTERNAL(WIKIPEDIA)]
Classical conditioning involves a neutral stimulus becoming a conditioned stimulus.
[Doc 2 | SOURCE=EXTERNAL(TEXTBOOK)]
Pavlov's dog experiment demonstrated classical conditioning by pairing a bell with food.

Question: Explain the mechanism of classical conditioning.

New Context:
[Doc 1 | SOURCE=MERGED(EXTERNAL) | FROM=1,2]
Classical conditioning is a learning process where a neutral stimulus (like a bell) becomes a conditioned stimulus after being paired with an unconditioned stimulus (like food). This was demonstrated in Pavlov's dog experiment.
</Example 2>

[Real Task]

Initial Context (numbered documents):
{context_init}

Last Context (may be empty):
{last_context}

Question:
{question_text}

Now produce your New Context following the examples.
""".strip()

p_con_prompt = ChatPromptTemplate.from_template(P_CON_TMPL)

def run_p_con(
    question_text: str,
    context_init: str,
    last_context: str = "",
    llm=None,
) -> str:
    if llm is None:
        try:
            llm = llm_solar_pro
        except NameError:
            llm = llm_solver

    msgs = p_con_prompt.format_messages(
        question_text=question_text,
        context_init=context_init,
        last_context=last_context,
    )
    resp = llm.invoke(msgs)
    return resp.content.strip()


In [ ]:
############################################
# 7. Knowledge Consolidation + Answer Finalization p_ans (10-Option Optimized)
############################################

import re

def extract_final_answer_letter(text: str) -> str:
    """
    모델이 내뱉는 다양한 형식을 모두 잡아내는 강력한 추출 함수
    """
    if not text: return ""

    # 1순위: "Final Answer" 뒤에 오는 첫 번째 영문자 (A-J) 찾기
    # 예: "**Final Answer:** (A)", "Final Answer: Option A", "Final Answer is [A]"
    # [^\nA-J]* : 줄바꿈이나 A-J가 나오기 전까지의 모든 특수문자(:, *, 공백 등)를 무시
    match = re.search(r"Final Answer[^\nA-J]*([A-J])", text, flags=re.IGNORECASE | re.MULTILINE)
    if match:
        return match.group(1).upper()

    # 2순위: <ANSWER> 태그 (프롬프트에서 지시한 경우)
    match_tag = re.search(r"<ANSWER>\s*\(?([A-J])\)?", text, flags=re.IGNORECASE)
    if match_tag:
        return match_tag.group(1).upper()

    # 3순위: 최후의 수단 - 문장 맨 끝에 있는 (A) 형태 찾기
    fallback = re.findall(r"\(?([A-J])\)?", text.split("\n")[-1])
    if fallback:
        return fallback[-1].upper()

    return "" # 정말 답이 없는 경우


In [ ]:
# ============================================
# 카테고리별 역할 / 풀이 전략 텍스트
# ============================================

CATEGORY_META = {
    "law": {
        "role": (
            "You are an expert law exam solver. "
            "You specialize in bar-exam style multiple-choice questions "
            "in criminal law, torts, contracts, constitutional law, property, and evidence."
        ),
        "guidelines": (
            "When solving LAW questions:\n"
            "- Treat the QUESTION as a fact pattern and map it to specific legal rules and elements.\n"
            "- Identify the relevant doctrine (e.g., mens rea, causation, standing, jurisdiction, remedies).\n"
            "- For each option, check whether ALL required elements are satisfied or if any element is missing.\n"
            "- If CONTEXT (textbook/Wikipedia) conflicts with the specific facts in the QUESTION, "
            "always follow the QUESTION's fact pattern.\n"
        ),
    },
    "history": {
        "role": (
            "You are an expert history exam solver. "
            "You handle world, U.S., European, Asian, Latin American and African history, "
            "and questions based on primary-source excerpts."
        ),
        "guidelines": (
            "When solving HISTORY questions:\n"
            "- First infer the approximate time period, region, and key actors from the QUESTION.\n"
            "- Use CONTEXT mainly to confirm chronology, causes, and consequences of events.\n"
            "- Pay attention to clues about ideology, institutions, wars, treaties, and social movements.\n"
            "- If the QUESTION's wording suggests a specific event or policy, treat that as the strongest clue.\n"
        ),
    },
    "philosophy": {
        "role": (
            "You are an expert philosophy exam solver. "
            "You handle ethics, epistemology, metaphysics, logic, philosophy of mind and language, "
            "political philosophy, and the history of philosophy."
        ),
        "guidelines": (
            "When solving PHILOSOPHY questions:\n"
            "- Identify which author, school, or position is being tested (e.g., Kantian, utilitarian, rationalist).\n"
            "- Pay close attention to technical distinctions (e.g., de re vs. de dicto, internalism vs. externalism).\n"
            "- Use CONTEXT to recall canonical definitions, standard objections, and typical examples.\n"
            "- Match each option to the most precise doctrinal description instead of relying on vague intuition.\n"
        ),
    },
    "psychology": {
        "role": (
            "You are an expert psychology exam solver. "
            "You handle cognitive, social, clinical, developmental, and biological psychology."
        ),
        "guidelines": (
            "When solving PSYCHOLOGY questions:\n"
            "- Identify the main subfield (e.g., memory, perception, conditioning, social cognition, development).\n"
            "- Pay attention to experimental design details (IV, DV, control, randomization) when present.\n"
            "- Use CONTEXT to recall definitions of constructs and classic findings.\n"
            "- Carefully distinguish between similar concepts (e.g., classical vs operant conditioning, "
            "state vs trait, positive vs negative reinforcement).\n"
        ),
    },
    "business": {
        "role": (
            "You are an expert business and economics exam solver. "
            "You handle microeconomics, macroeconomics, corporate finance, accounting, management, and marketing."
        ),
        "guidelines": (
            "When solving BUSINESS/ECON questions:\n"
            "- First identify the main domain (e.g., micro, macro, finance, accounting, management, marketing).\n"
            "- Extract the key variables and relationships (e.g., supply/demand shifts, elasticity, NPV, leverage).\n"
            "- Use CONTEXT to recall definitions, formulas, and standard qualitative implications.\n"
            "- When numbers are given, reason about direction and relative size; compute exactly only if necessary.\n"
        ),
    },
}

DEFAULT_CATEGORY_META = {
    "role": (
        "You are an expert exam solver for MMLU-Pro across law, history, philosophy, psychology, and business."
    ),
    "guidelines": (
        "Always prioritize the QUESTION text, use the CONTEXT as supporting evidence, "
        "and compare each option carefully against both."
    ),
}

In [ ]:
P_ANS_TMPL = """
You are an Astute-RAG style expert exam solver for MMLU-Pro.

[Category Specialization]
{category_role}

[Category-Specific Strategy]
{category_guidelines}

Your goal:
- Use BOTH your INTERNAL memorized knowledge and EXTERNAL retrieved documents.
- Always prioritize the QUESTION constraints over conflicting evidence.

[QUESTION]
{question_with_options}

Initial Context:
{context_init}

[Consolidated Context]:
{context_con}

Astute-RAG Reasoning Procedure:

Step 1. Analyze the Question
- Identify the core concept and any specific constraints (dates, negation, strict conditions).
- If the question asks for the "BEST" option, define the criteria for "best".

Step 2. Process of Elimination (CRITICAL)
- For EACH option (A) through (J), explicitly check if it contradicts the Consolidated Context or the Question constraints.
- Mark options as [ELIMINATED] if they are factually wrong or irrelevant.
- Mark options as [POSSIBLE] if they are supported by evidence.

Step 3. Pairwise Comparison & Differentiation (CRITICAL UPDATE)
- Look at the options marked [POSSIBLE].
- Compare them in pairs (e.g., Option A vs. Option B).
- Identify the **ONE distinguishing factor** that separates them (e.g., "Option A applies to Stage 1, while Option B applies to Stage 2").
- Check this distinguishing factor against the Question's constraints.
- If an option fits the context generally but fails the specific distinguishing factor required by the question, ELIMINATE it.

Step 4. The "Devil's Advocate" Check
- Pick your tentative best answer (e.g., Option B).
- Ask yourself: "Why might Option B be WRONG?" (Look for subtle traps, strict legal definitions, or exceptions).
- If the counter-argument is weak, confirm Option B. If it's strong, reconsider the runner-up.

Final Output Format:
Provide your step-by-step reasoning, and then end with exactly:
Final Answer: X
(where X is the capital letter of the correct option)
""".strip()

p_ans_prompt = ChatPromptTemplate.from_template(P_ANS_TMPL)

In [ ]:
# 살짝 변경
def run_p_ans(
    context_init: str,
    context_con: str,
    full_question: str,
    category: str,
    llm=None,
) -> Dict[str, Any]:
    """
    p_gen/p_con으로 만들어진 context를 기반으로,
    astute-RAG 방식(INTERNAL vs EXTERNAL, Step 1~4)을 그대로 따르되,
    category별 역할/전략 텍스트를 추가하여 특화된 solver로 동작하게 함.
    """
    if llm is None:
        try:
            llm = llm_solar_pro
        except NameError:
            llm = llm_solver

    meta = CATEGORY_META.get(category, DEFAULT_CATEGORY_META)

    msgs = p_ans_prompt.format_messages(
        category_role=meta["role"],
        category_guidelines=meta["guidelines"],
        context_init=context_init,
        context_con=context_con,
        question_with_options=full_question,
    )
    resp = llm.invoke(msgs)
    raw = resp.content.strip()
    letter = extract_final_answer_letter(raw)
    return {"final_answer": letter, "raw_reasoning": raw}

In [ ]:
from collections import Counter
from langchain_upstage import ChatUpstage # 혹시 import 안 되어 있을까봐

def solve_mmlu_astute_style(
    full_prompt: str,
    use_wiki: bool = True,
    n_vote: int = 5  # [설정] 투표 횟수 (보통 3~5 추천)
) -> Dict[str, Any]:

    # ---------------------------------------------------------
    # 1. Analyze & Retrieval (기존과 동일)
    # ---------------------------------------------------------
    # 1) 질문/선지 분리
    question_text, options = parse_question_and_options(full_prompt)

    # 2) 질문 분석 (p_analyze)
    # (앞서 추가한 run_p_analyze 함수가 있다고 가정)
    try:
        analysis = run_p_analyze(question_text, options, llm=llm_solver)
        constraints = analysis.get("constraints", [])
        search_queries = analysis.get("search_queries", [question_text])
    except NameError:
        # p_analyze를 아직 정의 안 했다면 기본값 처리
        analysis = {}
        constraints = []
        search_queries = [question_text]

    print(f"   -> [Analysis] Constraints: {constraints[:3]}")

    # 3) Category 분류
    category, extracted_keywords = classify_and_extract_keywords(question_text)
    if category not in vectorstore:
        category = "business"

    # 4) Retrieval (분석된 쿼리 사용)
    combined_query = " ".join(search_queries)
    tb_passages = get_textbook_passages(category, combined_query)

    # 위키 검색 (기존 키워드 방식 유지 혹은 쿼리 활용)
    # 여기서는 호환성을 위해 기존 로직 + 쿼리 활용
    wiki_passages = []
    if use_wiki:
        # 쿼리에서 명사만 추출하거나, 분석된 쿼리를 사용해 위키 검색
        # (편의상 기존 get_wiki_passages 사용 가정)
        # Use extracted_keywords from classification for wiki search
        wiki_passages = get_wiki_passages(extracted_keywords)

    # ---------------------------------------------------------
    # 2. Context Building (p_mem -> p_gen -> p_con) (기존과 동일)
    # ---------------------------------------------------------
    # 5) p_mem: INTERNAL 문서
    doc_blocks = []
    next_doc_idx = 1

    mem_doc_text = run_p_mem(category, question_text)
    if mem_doc_text.strip():
        doc_blocks.append(
            f"[Doc {next_doc_idx} | SOURCE=INTERNAL(MEMORY)]\n{mem_doc_text}"
        )
        next_doc_idx += 1

    # 6) p_gen: TEXTBOOK docs (LLM 없이 Pass-through)
    tb_block, next_doc_idx = run_p_gen_for_docs(
        category=category,
        source_type="TEXTBOOK",
        question_text=question_text,
        raw_passages=tb_passages,
        start_doc_idx=next_doc_idx,
        llm=None, # LLM 사용 안 함
    )
    if tb_block.strip():
        doc_blocks.append(tb_block)

    # 7) p_gen: WIKIPEDIA docs
    if use_wiki and wiki_passages:
        wiki_block, next_doc_idx = run_p_gen_for_docs(
            category=category,
            source_type="WIKIPEDIA",
            question_text=question_text,
            raw_passages=wiki_passages,
            start_doc_idx=next_doc_idx,
            llm=None,
        )
        if wiki_block.strip():
            doc_blocks.append(wiki_block)

    context_init = "\n\n".join(doc_blocks)

    # 8) p_con: consolidation
    context_con = run_p_con(
        question_text=question_text,
        context_init=context_init,
        last_context="",
    )

    # ---------------------------------------------------------
    # 3. Voting (Self-Consistency) ★ [여기가 변경됨]
    # ---------------------------------------------------------

    # 분석된 제약조건을 질문에 포함
    constraints_str = "\n".join([f"- {c}" for c in constraints])
    augmented_question = f"{full_prompt}\n\n[CRITICAL CONSTRAINTS]\n{constraints_str}"

    # 투표용 LLM (Temperature를 높여서 다양성 확보)
    llm_voter = ChatUpstage(
        api_key=UPSTAGE_API_KEY,
        model="solar-pro2",
        temperature=0.7  # 0.5 ~ 0.8 사이 추천
    )

    votes = []
    reasonings = []

    print(f"   -> [Voting] Running {n_vote} iterations...")

    for i in range(n_vote):
        try:
            # p_ans 실행
            ans = run_p_ans(
                context_init=context_init,
                context_con=context_con,
                full_question=augmented_question,
                category=category,
                llm=llm_voter # Voter LLM 사용
            )

            letter = ans["final_answer"]
            # 유효한 답(A-J)만 투표에 포함
            if letter and letter in "ABCDEFGHIJ":
                votes.append(letter)

            reasonings.append(f"[Run {i+1} Answer: {letter}]\n{ans['raw_reasoning']}")

        except Exception as e:
            print(f"      [Error in vote {i}]: {e}")
            continue

    # ---------------------------------------------------------
    # 4. Result Aggregation
    # ---------------------------------------------------------
    if not votes:
        final_ans = "A" # 투표 실패 시 기본값 (혹은 에러 처리)
        print("   -> [Warn] No valid votes collected.")
    else:
        # 최빈값(Mode) 찾기
        count = Counter(votes)
        final_ans = count.most_common(1)[0][0]

        # 투표 현황 로그 출력 (디버깅용)
        vote_summary = str(dict(count))
        print(f"   -> [Result] Winner: {final_ans} | Votes: {vote_summary}")

    # Reasoning 로그 합치기 (나중에 분석용)
    full_log = "\n" + "="*40 + "\n".join(reasonings)

    return {
        "final_answer": final_ans,
        "category": category,
        "analysis": analysis,
        "keywords": extracted_keywords, # ADDED THIS LINE
        "context_init": context_init,
        "context_con": context_con,
        "raw_reasoning": full_log, # 모든 투표의 Reasoning을 저장
        "vote_summary": str(dict(Counter(votes))) if votes else "None"
    }

In [ ]:
import pandas as pd

df = pd.read_csv("/content/testset_100.csv")
QUESTION_COL = "prompts"

results = []
for i, row in df.iterrows():
    q = row[QUESTION_COL]
    print(f"[{i}] Solving: {q[:60]}...")
    out = solve_mmlu_astute_style(q)
    results.append(out["final_answer"])
    df.loc[i, "pred_category"] = out["category"]
    df.loc[i, "kw1"] = out["keywords"][0] if len(out["keywords"]) > 0 else ""
    df.loc[i, "kw2"] = out["keywords"][1] if len(out["keywords"]) > 1 else ""
    df.loc[i, "kw3"] = out["keywords"][2] if len(out["keywords"]) > 2 else ""
    df.loc[i, "rag_cot_answer"] = out["final_answer"]
    df.loc[i, "cot_full"] = out["raw_reasoning"]

df.to_csv("astute-rag-1126-final.csv", index=False)
print("저장 완료:1126.csv")

[0] Solving: QUESTION4868) In 1797, John Frere made a discovery that he d...
   -> [Analysis] Constraints: ['Must involve John Frere', 'Must be from 1797', 'Must describe his discovery']
[Init] Building BM25 index for: history...
[Init] BM25 ready for history (17941 docs)
   -> [Voting] Running 5 iterations...
   -> [Result] Winner: I | Votes: {'I': 5}
[1] Solving: QUESTION586) BobGafneyand Susan Medina invested $40,000 and ...
   -> [Analysis] Constraints: ['Bob Gafney invested $40,000 and receives 4% of net income as manager', 'Susan Medina invested $50,000', 'Each partner receives 6% interest on their investment']
[Init] Building BM25 index for: business...
[Init] BM25 ready for business (6285 docs)
   -> [Voting] Running 5 iterations...
   -> [Result] Winner: F | Votes: {'F': 4, 'C': 1}
[2] Solving: QUESTION138) If at the beginning of each month a deposit of ...
   -> [Analysis] Constraints: ['Deposits of $500 made monthly', 'Interest rate of 8% compounded monthly', 'Time period of

In [ ]:
import pandas as pd

# 1. 파일 경로
GT_PATH = "/content/testset_100.csv"        # 정답 파일
PRED_PATH = "/content/drive/MyDrive/nlp team project/astute-rag-1126-final.csv"       # 출력 파일

# 2. 컬럼 이름
GT_COL = "answers"               # testset.csv에서 정답 컬럼
PRED_COL = "rag_cot_answer"      # baseline.csv에서 예측 컬럼

# 3. csv 로드
gt_df = pd.read_csv(GT_PATH)
pred_df = pd.read_csv(PRED_PATH)

# 4. Series 추출
gt = gt_df[GT_COL].astype(str)
pred = pred_df[PRED_COL].astype(str)

# 5. 정규화 함수
def normalize_choice(x: str) -> str:
    x = x.strip().upper()
    for ch in x:
        if "A" <= ch <= "Z":
            return ch
    return x

# 인덱스 리셋이 포인트
gt_norm = gt.apply(normalize_choice).reset_index(drop=True)
pred_norm = pred.apply(normalize_choice).reset_index(drop=True)

print(len(gt_norm), len(pred_norm))  # 둘 다 25 나오는지 체크 한번 해보고

# 6. 정확도 계산
correct = (gt_norm == pred_norm)
accuracy = correct.mean()

print(f"총 문제 수: {len(gt_norm)}")
print(f"정답 개수: {correct.sum()}")
print(f"정확도: {accuracy * 100:.2f}%")

100 100
총 문제 수: 100
정답 개수: 66
정확도: 66.00%


기존 testset.csv

In [ ]:
import pandas as pd
from langchain_core.prompts import ChatPromptTemplate


df = pd.read_csv("testset.csv")[25:]  # 질문, 선택지 다 합쳐진 컬럼 가정
QUESTION_COL = "prompts"

results = []
for i, row in df.iterrows():
    q = row[QUESTION_COL]
    print(f"[{i}] Solving: {q[:60]}...")
    out = solve_mmlu_astute_style(q)
    results.append(out["final_answer"])
    df.loc[i, "pred_category"] = out["category"]
    df.loc[i, "kw1"] = out["keywords"][0] if len(out["keywords"]) > 0 else ""
    df.loc[i, "kw2"] = out["keywords"][1] if len(out["keywords"]) > 1 else ""
    df.loc[i, "kw3"] = out["keywords"][2] if len(out["keywords"]) > 2 else ""
    df.loc[i, "rag_cot_answer"] = out["final_answer"]
    df.loc[i, "cot_full"] = out["raw_reasoning"]

df.to_csv("baseline-1126-llmreranker.csv", index=False)
print("저장 완료:baseline-1126.csv")

[25] Solving: QUESTION26) QUESTION 6) A psychologist is asked to see a 10-...
   -> [Analysis] Constraints: ['Must involve a 10-year-old child', 'Must occur in a school setting', 'Must adhere to ethical responsibilities for psychologists']
[Init] Building BM25 index for: psychology...
[Init] BM25 ready for psychology (2232 docs)
   -> [Voting] Running 5 iterations...
   -> [Result] Winner: D | Votes: {'D': 4, 'A': 1}
[26] Solving: QUESTION27) A man is at home in his apartment, alone, late a...
   -> [Analysis] Constraints: ['Must involve common law crimes', 'Must consider breaking into the apartment (burglary)', 'Must consider threat with a gun during theft (robbery)']
[Init] Building BM25 index for: law...
[Init] BM25 ready for law (91859 docs)
   -> [Voting] Running 5 iterations...
   -> [Result] Winner: A | Votes: {'A': 5}
[27] Solving: QUESTION28) What do Homo sapiens and Australopithecus afaren...
   -> [Analysis] Constraints: ['Answer must be one of the provided options (A-J)', '

In [ ]:
import pandas as pd

# 1. 파일 경로
GT_PATH = "testset.csv"          # 정답 파일
PRED_PATH = "baseline-1126-llmreranker.csv"  # 출력 파일 (현재 경로에 저장됨)

# 2. 컬럼 이름
GT_COL = "answers"               # testset.csv에서 정답 컬럼
PRED_COL = "rag_cot_answer"      # baseline.csv에서 예측 컬럼

# 3. csv 로드
gt_df = pd.read_csv(GT_PATH)[25:]
pred_df = pd.read_csv(PRED_PATH)

# 4. Series 추출
gt = gt_df[GT_COL].astype(str)
pred = pred_df[PRED_COL].astype(str)

# 5. 정규화 함수
def normalize_choice(x: str) -> str:
    x = x.strip().upper()
    for ch in x:
        if "A" <= ch <= "Z":
            return ch
    return x

# 인덱스 리셋 (중요: 길이를 맞추기 위해)
gt_norm = gt.apply(normalize_choice).reset_index(drop=True)
pred_norm = pred.apply(normalize_choice).reset_index(drop=True)

print(f"Ground Truth 개수: {len(gt_norm)}, Prediction 개수: {len(pred_norm)}")

# 6. 정확도 계산
correct = (gt_norm == pred_norm)
accuracy = correct.mean()

print(f"총 문제 수: {len(gt_norm)}")
print(f"정답 개수: {correct.sum()}")
print(f"정확도: {accuracy * 100:.2f}%")

Ground Truth 개수: 25, Prediction 개수: 25
총 문제 수: 25
정답 개수: 20
정확도: 80.00%
